In [3]:
import os
from langchain_openai.chat_models.azure import AzureChatOpenAI

openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
openai_api_base = os.getenv('AZURE_OPENAI_ENDPOINT')
openai_api_version = os.getenv('AZURE_OPENAI_API_VERSION')
embeddings_model = os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME')
gpt_model = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

llm = AzureChatOpenAI(
                openai_api_version=openai_api_version,
                azure_deployment=gpt_model,
            )

In [ ]:
request = """we're participating in a hackaton event about gen ai. 
Our project is about creating an app that implements 
- generative ai
- langgraph
- react
- tailwind
- rag architecture

to generate different kind of files that can be used by new joiners or new team members for self study.

please generate a list of 15 smart and cool names that we can use in the hackaton for naming our project"""

output = llm.invoke(request)
print(output.content)

Here are 15 creative and cool names for your generative AI app designed for new joiners and team members:

1. **OnboardGen**
2. **LearnFlow AI**
3. **SkillWave**
4. **GenieGuide**
5. **TeamLeap**
6. **KnowledgeNest**
7. **InsightForge**
8. **FlowMinds**
9. **GenToolkit**
10. **CurioCraft**
11. **TeamTrove**
12. **InfoPulse**
13. **AccelaLearn**
14. **TandemThink**
15. **EduFusion AI**

Feel free to mix and match or modify these names to better suit your project’s vision! Good luck with your hackathon!


: 

In [7]:

from langchain_core.prompts import PromptTemplate

def generate_ppt_request(context: str):
    "generate a sample request for creating a ppt file using pptx library"

    prompt = """You are a smart assistant than can create payload samples for creating pptx files.
        The content of the sample must be extracted from this context:

        ### Context ###
        {context}
        Your job is to generate a sample request following the example below.
        

        The title field is the title of the document.
        The pages field contain a list of the different slides and their content.
        The TextItem contains two properties: type and content
            the type is a string and can have the following values: "title", "subtitle", "paragraph" 
            the content a string and represents the content of the text item
        
        The DocumentContent object is a list of TextItems.
        
        Please generate a sample request based on the context above. feel free to decide how many slides, the titles, paragraphs and content
        of each slide.

        ### Example of a request ###
        DocumentRequest(
            title="Demo Document",
            pages=[
                DocumentContent(
                    text_items=[
                        TextItem(type="header", content="Welcome"),
                        TextItem(type="paragraph", content="This is a sample slide."),
                    ],
                ),

                DocumentContent(
                    text_items=[
                        TextItem(type="subheader", content="Second Slide"),
                        TextItem(type="paragraph", content="Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vivamus luctus urna sed urna ultricies ac tempor dui sagittis. In condimentum facilisis porta.\nFusce sed felis eget velit aliquet faucibus. Praesent ac massa at ligula laoreet iaculis."),
                    ],  
                )
            ]
        )

        Return just the sample payload. Nothing else.
        """

    prompt_template = PromptTemplate.from_template(prompt)
    chain  = prompt_template | llm
    sample_request = chain.invoke({"context": context})
    return sample_request

In [8]:
sample = generate_ppt_request("there are several animals in the jungle, such as the lion, tiger and elephant. funny huh ?")

In [12]:
def clean_request(request:str):
        return request.replace("python","").replace("`", "").replace("json","")

print(clean_request(sample.content))

req = clean_request(sample.content)


{
    "title": "Jungle Animals",
    "pages": [
        {
            "text_items": [
                {
                    "type": "title",
                    "content": "Exploring the Jungle"
                },
                {
                    "type": "paragraph",
                    "content": "The jungle is home to a diverse array of animals such as lions, tigers, and elephants."
                }
            ]
        },
        {
            "text_items": [
                {
                    "type": "subtitle",
                    "content": "Majestic Lions"
                },
                {
                    "type": "paragraph",
                    "content": "Lions are known as the kings of the jungle, recognized for their strength and pride."
                }
            ]
        },
        {
            "text_items": [
                {
                    "type": "subtitle",
                    "content": "Powerful Tigers"
                },
                

In [13]:
from pptx import Presentation
from pptx.enum.text import MSO_AUTO_SIZE
from pptx.util import Inches, Pt


TEXT_BLOCK_HEIGHT = Inches(0.8)
IMAGE_HEIGHT = Inches(2)
IMAGE_SPACING = Inches(0.2)
MAX_CONTENT_HEIGHT = Inches(7.0)
LOCAL_IMG_DIR="../assets/generated_images/2025-04-21-11-57-47"
def generate_ppt(doc_data, output_path: str):
        prs = Presentation()

        for slide_data in doc_data.pages:
            slide = prs.slides.add_slide(prs.slide_layouts[5])

            # Customizing text dimensions
            num_blocks = len(slide_data.text_items or [])
            text_height_estimate = num_blocks * TEXT_BLOCK_HEIGHT
            textbox = slide.shapes.add_textbox(Inches(0.5), Inches(0.5), Inches(8.5), text_height_estimate)
            tf = textbox.text_frame
            tf.word_wrap = True
            tf.auto_size = None#MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE

            for i, element in enumerate(slide_data.text_items or []):
                para = tf.paragraphs[0] if i == 0 else tf.add_paragraph()
                para.text = element.content
                para.space_after = Pt(10)
                font = para.font

                if element.type == "title":
                    font.size = Pt(32)
                    font.bold = True
                elif element.type == "subtitle":
                    font.size = Pt(24)
                    font.italic = True
                elif element.type == "paragraph":
                    font.size = Pt(16)
                else:
                    font.size = Pt(14)

            # Customizing images
            image_top = Inches(0.5) + text_height_estimate + Inches(0.5)

            for img_name in slide_data.images or []:
                image_path = os.path.join(LOCAL_IMG_DIR, img_name)
                if os.path.exists(image_path):
                    # Check if this image fits on the current slide
                    if image_top + IMAGE_HEIGHT > MAX_CONTENT_HEIGHT:
                        # Create a new slide and reset
                        slide = prs.slides.add_slide(prs.slide_layouts[5])
                        image_top = Inches(0.5)

                    try:
                        slide.shapes.add_picture(image_path, Inches(0.5), image_top, height=IMAGE_HEIGHT)
                        image_top += IMAGE_HEIGHT + IMAGE_SPACING
                    except Exception as e:
                        print(f"❌ Error adding new image: {e}")

In [15]:
generate_ppt(req, "../api/output")

AttributeError: 'str' object has no attribute 'pages'